# Document Extraction with AI_EXTRACT
Create stage and upload PDF files:

In [ ]:
%%sql -r dataframe_1
CREATE SCHEMA IF NOT EXISTS AI_DEMOS.AI_FUNCTIONS_DEMO;
USE SCHEMA AI_DEMOS.AI_FUNCTIONS_DEMO;

CREATE STAGE IF NOT EXISTS TICKETS 
	DIRECTORY = ( ENABLE = true ) 
	ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

In [ ]:
from snowflake.snowpark.context import get_active_session

session = get_active_session()

session.file.put('tickets/*', '@AI_DEMOS.AI_FUNCTIONS_DEMO.TICKETS', auto_compress=False, overwrite=True)
_ = session.sql("ALTER STAGE TICKETS REFRESH").collect()

## AI_EXTRACT

In [ ]:
%%sql -r dataframe_3
CREATE OR REPLACE TABLE TICKETS_EXTRACTED AS 
SELECT
  -- Extract information from documents
  AI_EXTRACT(
    file => TO_FILE('@AI_DEMOS.AI_FUNCTIONS_DEMO.TICKETS', RELATIVE_PATH),
    responseFormat => PARSE_JSON('{
        "schema": {
            "type": "object",
            "properties": {
                "arrival_date": {
                    "description": "What is the arrival date?",
                    "type": "string"
                },
                "departure_date": {
                    "description": "What is the departure date?",
                    "type": "string"
                },
                "destination": {
                    "description": "What is the destination station?",
                    "type": "string"
                },
                "origin": {
                    "description": "What is the origin station?",
                    "type": "string"
                },
                "ticket_class": {
                    "description": "What is the ticket class?",
                    "type": "string"
                },
                "ticket_number": {
                    "description": "What is the order number?",
                    "type": "string"
                },
                "ticket_price": {
                    "description": "What is the total ticket price? Return the price without currency in with . for decimals.",
                    "type": "string"
                },
                "train_numbers": {
                    "description": "List all train numbers from the ticket",
                    "type": "array"
                }
            }
        }
    }')
) AS extracted_data,

  -- get structured outputs
  extracted_data['response']['ticket_number']::TEXT TICKET_NUMBER,
  TRY_TO_DATE(extracted_data['response']['arrival_date']::TEXT, 'DD.MM.YYYY') ARRIVAL_DATE,
  TRY_TO_DATE(extracted_data['response']['departure_date']::TEXT, 'DD.MM.YYYY') DEPARTURE_DATE,
  extracted_data['response']['origin']::TEXT ORIGIN,
  extracted_data['response']['destination']::TEXT DESTINATION,
  extracted_data['response']['ticket_class']::TEXT TICKET_CLASS,
  extracted_data['response']['ticket_price']::FLOAT TICKET_PRICE,
  extracted_data['response']['train_numbers']::ARRAY TRAIN_NUMBERS
FROM
  DIRECTORY('@AI_DEMOS.AI_FUNCTIONS_DEMO.TICKETS');

SELECT * FROM TICKETS_EXTRACTED;

In [ ]:
%%sql -r dataframe_6
SELECT 
  YEAR(DEPARTURE_DATE) TICKET_YEAR,
  MONTH(DEPARTURE_DATE) TICKET_MONTH,
  COUNT(*) TOTAL_TICKETS, 
  SUM(TICKET_PRICE) TOTAL_SUM, 
  AVG(TICKET_PRICE) AVERAGE_TICKET_PRICE
FROM TICKETS_EXTRACTED
  GROUP BY TICKET_YEAR, TICKET_MONTH
  ORDER BY TICKET_YEAR, TICKET_MONTH;